## Make a static maps

### Download libraries and dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np

from matplotlib.colors import Normalize,LogNorm
#######################################################

filename = '../data/cleaned/cleaned_dataset_v3.csv'

shp_name = '../shapefiles/provinces.shp'

#results_dir = './'
results_dir = '../map_visualization_results/'

#######################################################

df = pd.read_csv(filename,delimiter=',')

col = 'url'  # Drop the column Url because it needn't for analysis
if col in df.columns:
    df.drop(columns=[col], inplace=True)

## Add province column

In [2]:
def get_province(pcode: int):
    """
    Convert belgium postal code to the name of province (and abstract number)
    """
       
    if 1000 <= pcode <= 1299:
        return 10, 'Brussels'
    elif 1300 <= pcode <= 1499:
        return 1,'Walloon Brabant'
    elif 1500 <= pcode <= 1999 or 3000 <= pcode <= 3499:
        return 3, 'Flemish Brabant'
    elif 2000 <= pcode <= 2999:
        return 2, 'Antwerp'
    elif 3500 <= pcode <= 3999:
        return 9, 'Limburg'
    elif 4000 <= pcode <= 4999:
        return 4, 'Liege'
    elif 5000 <= pcode <= 5999:
        return 5, 'Namur'
    elif 6000 <= pcode <= 6599 or 7000 <= pcode <= 7999:
        return 7, 'Hainaut'
    elif 6600 <= pcode <= 6900:
        return 6, 'Luxembourg'
    elif 8000 <= pcode <= 8999:
        return 8, 'West Flanders'
    elif 9000 <= pcode <= 9999:
        return 9, 'East Flanders'
    else:
        return 0,'Incorrect postal code'

In [3]:

df['province_code'] = df['postal_code'].apply(lambda x: get_province(x)[0])

df['province_name'] = df['postal_code'].apply(lambda x: get_province(x)[1])


## Create cdata for maps

In [4]:
HIGH_VALUE_PROP_PRICE = 500_000
HIGH_VALUE_PROP_PRICE_PER_M2 = 5000

df_count = df.groupby('province_code').size().reset_index(name='count')

percent_luxurious = (
    df.groupby('province_code')['is_luxurious']
    .mean()  # mean - 
    .mul(100)  # convert to percents
    .astype(int)
    .reset_index(name='percent_luxurious')
)

price_variance = df.groupby('province_code')['price'].var().reset_index(name='price_variance')

median_area = df.groupby('province_code')['area'].median().reset_index(name='median_area')

median_price = df.groupby('province_code')['price'].median().reset_index(name='median_price')
mean_price = df.groupby('province_code')['price'].mean().reset_index(name='mean_price')

df['price_per_m2'] = df['price'] / df['area']

df_price_per_m2 = df.groupby('province_code', as_index=False)['price_per_m2'].median()
df_price_per_m2.rename(columns={'price_per_m2': 'median_price_per_m2'}, inplace=True)

df_price_per_m2_average = df.groupby('province_code', as_index=False)['price_per_m2'].mean()
df_price_per_m2_average.rename(columns={'price_per_m2': 'mean_price_per_m2'}, inplace=True)

######################################################################################

df_summary = pd.merge(df_count, percent_luxurious, on='province_code', how='left')
df_summary = df_summary.merge(price_variance, on='province_code', how='left')
df_summary = df_summary.merge(median_area, on='province_code', how='left')
df_summary = df_summary.merge(median_price, on='province_code', how='left')
df_summary = df_summary.merge(df_price_per_m2, on='province_code', how='left')
df_summary = df_summary.merge(mean_price, on='province_code', how='left')
df_summary = df_summary.merge(df_price_per_m2_average, on='province_code', how='left')



## prepare shapefile, join it with data

In [5]:
gdf = gpd.read_file(shp_name)

gdf['prov_num'] = pd.to_numeric(gdf['prov_num'], errors='coerce').astype('Int64')
df_summary['province_code'] = pd.to_numeric(df_summary['province_code'], errors='coerce').astype('Int64')

gdf = gdf.merge(df_summary, left_on='prov_num', right_on='province_code', how='left')

print( gdf.info() )

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   prov_num             12 non-null     Int64   
 1   prov_name            12 non-null     object  
 2   Join_Count           12 non-null     int32   
 3   nouveau_PO           12 non-null     object  
 4   FREQUENCY            12 non-null     int32   
 5   CP_speciau           12 non-null     int32   
 6   Shape_Leng           12 non-null     float64 
 7   Shape_Area           12 non-null     float64 
 8   geometry             12 non-null     geometry
 9   province_code        12 non-null     Int64   
 10  count                12 non-null     int64   
 11  percent_luxurious    12 non-null     int64   
 12  price_variance       12 non-null     float64 
 13  median_area          12 non-null     float64 
 14  median_price         12 non-null     float64 
 15  median_price_per_

## Load some specific libraries

In [6]:
import folium
import geopandas as gpd
import branca.colormap as cm
from folium.features import GeoJsonTooltip

### Create interactive map

In [7]:
tooltip_fields = [
    'prov_name',
    'count',
    'percent_luxurious',
    'median_area',
    'median_price',
    'mean_price',
    'median_price',
    'mean_price'
]

tooltip_aliases = [
    'Province',
    'Number of properties',
    'Luxurious, %',
    'Median Area (sq.m.)',
    'Median Price (€)',
    'Average Price (€)',
    'Price per m2 (€)',
    'Average price / m2'
]


In [8]:
# Function for adding layer by a variable
def add_variable_layer(gdf, variable, colormap, layer_name,show=False):
    layer = folium.FeatureGroup(name=layer_name,show=show)

    common_tooltip = GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=tooltip_aliases,
        localize=True,
        sticky=True
    )
    
    folium.GeoJson(
        gdf,
        style_function=lambda feature: {
            'fillColor': colormap(feature['properties'][variable])
            if feature['properties'][variable] is not None else 'transparent',
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7
        },
        tooltip=common_tooltip
    ).add_to(layer)
    layer.add_to(fmap)
    return layer

In [9]:
# Create folium map
fmap = folium.Map(location=[50.85, 4.35], zoom_start=8, tiles=None)

# Create scales

##################################################################
##################################################################
# 'Blues_09' — светло-голубой → тёмно-синий
# 'Greens_09' — светло-зелёный → тёмно-зелёный
# 'Reds_09' — розовый → бордовый
# 'OrRd_09' — оранжево-красный
# 'PuBu_09' — пурпурно-синий
# 'BuGn_09' — голубой → зелёный
# 'YlGnBu_09' — жёлто-зелёно-синий
# 'YlOrBr_09' — жёлто-оранжево-коричневый
##################################################################

colormap_count = cm.linear.Blues_09.scale(0, gdf['count'].max())
colormap_count.caption = 'Number of properties'
add_variable_layer(gdf, 'count', colormap_count, 'Number of properties')
colormap_count.add_to(fmap)

colormap_luxurious = cm.linear.Reds_09.scale(gdf['percent_luxurious'].min(),gdf['percent_luxurious'].max())
colormap_luxurious.caption = 'Percent of luxurious Prop.'
add_variable_layer(gdf, 'percent_luxurious', colormap_luxurious, 'Percent of luxurious Prop.')
colormap_luxurious.add_to(fmap)

colormap_area = cm.linear.Reds_09.scale(gdf['median_area'].min(),gdf['median_area'].max())
colormap_area.caption = 'Median Area (sq.m.)'
add_variable_layer(gdf, 'median_area', colormap_area, 'Median Area (sq.m.)')
colormap_area.add_to(fmap)

colormap_price = cm.linear.PuBu_09.scale(200_000,400_000)
colormap_price.caption = 'Median Price (€)'
add_variable_layer(gdf, 'median_price', colormap_price, 'Median Price (€)',True)
colormap_price.add_to(fmap)

colormap_price = cm.linear.PuBu_09.scale(200_000,600_000)
colormap_price.caption = 'Average Price (€)'
add_variable_layer(gdf, 'mean_price', colormap_price, 'Average Price (€)')
colormap_price.add_to(fmap)

colormap_per_m2 = cm.linear.Greens_09.scale(1000,4000)
colormap_per_m2.caption = 'Price per m2 (€)'
add_variable_layer(gdf, 'median_price_per_m2', colormap_per_m2, 'Price per m2')
colormap_per_m2.add_to(fmap)

colormap_per_m2 = cm.linear.Greens_09.scale(1000,4000)
colormap_per_m2.caption = 'Average Price / m2'
add_variable_layer(gdf, 'median_price_per_m2', colormap_per_m2, 'Average Price / m2')
colormap_per_m2.add_to(fmap)

##################################################################
##################################################################

folium.LayerControl(collapsed=False).add_to(fmap)

#######################################################
# Save the map into HTML file

outname = 'interactive_provinces_map_v3.html'

filepath = os.path.join(results_dir, outname)

fmap.save(filepath)

# Show the map in JupNotebook
#fmap